install tensorflow

In [ ]:
!pip install tensorflow

importing tensorflow,sklearn and pandas libraries

In [ ]:
import tensorflow as tf
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
#i dont think tensorflow has a train test split library so i had to borrow it from sklearn
from sklearn.model_selection import train_test_split
#used the same evaluation metrics for the sake of fairness
from sklearn.metrics import accuracy_score, precision_score, f1_score

reading and preprocessing csv file from null values if there was any

In [ ]:
dp = pd.read_csv('/content/sentiment.csv')

dp.fillna('none', inplace=True)
dp.to_csv("clean_data.csv", index=False)
dp.describe()


,sentiment
count,5000.00000
mean,0.50000
std,0.50005
min,0.00000
25%,0.00000
50%,0.50000
75%,1.00000
max,1.00000


dividing data into training and testing 20% for testing 80% for training

In [ ]:
X = dp['review']
Y = dp['sentiment']

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2,random_state = 0)

tokenizing data and using texts_to_sequences to turn text into numbers

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)
X_train_tokens = tokenizer.texts_to_sequences(X_train)
X_test_tokens = tokenizer.texts_to_sequences(X_test)

max_sequence_length = max(len(tokens) for tokens in X_train_tokens)
X_train_pad = pad_sequences(X_train_tokens, maxlen=max_sequence_length)
X_test_pad = pad_sequences(X_test_tokens, maxlen=max_sequence_length)

building neural network model

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=100, input_length=max_sequence_length),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(1, activation='sigmoid')
])


compile and train the model

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train_pad, y_train, epochs=10, batch_size=16, validation_data=(X_test_pad, y_test))

Epoch 1/10
250/250 [==============================] - 5s 17ms/step - loss: 0.6830 - accuracy: 0.6040 - val_loss: 0.6677 - val_accuracy: 0.6750
Epoch 2/10
250/250 [==============================] - 4s 16ms/step - loss: 0.6225 - accuracy: 0.7542 - val_loss: 0.6136 - val_accuracy: 0.6450
Epoch 3/10
250/250 [==============================] - 4s 14ms/step - loss: 0.5122 - accuracy: 0.8413 - val_loss: 0.5216 - val_accuracy: 0.8130
Epoch 4/10
250/250 [==============================] - 4s 15ms/step - loss: 0.4040 - accuracy: 0.8953 - val_loss: 0.4657 - val_accuracy: 0.8310
Epoch 5/10
250/250 [==============================] - 4s 16ms/step - loss: 0.3212 - accuracy: 0.9178 - val_loss: 0.4267 - val_accuracy: 0.8380
Epoch 6/10
250/250 [==============================] - 4s 15ms/step - loss: 0.2604 - accuracy: 0.9370 - val_loss: 0.4027 - val_accuracy: 0.8380
Epoch 7/10
250/250 [==============================] - 4s 15ms/step - loss: 0.2131 - accuracy: 0.9535 - val_loss: 0.3875 - val_accuracy: 0.8410

make prediction

In [ ]:
y_pred_train = model.predict(X_train_pad)
y_pred_train_classes = (y_pred_train > 0.5).astype(int)

y_pred_test = model.predict(X_test_pad)
y_pred_test_classes = (y_pred_test > 0.5).astype(int)

32/32 [==============================] - 0s 1ms/step


evaluation metrics

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, f1_score

accuracy = accuracy_score(y_test, y_pred_test_classes)
precision = precision_score(y_test, y_pred_test_classes, average='weighted')
f1 = f1_score(y_test, y_pred_test_classes, average='weighted')

print("Accuracy: ", accuracy)
print("Precision: ", precision)
print("F1-score: ", f1)

Accuracy:  0.846
Precision:  0.8462584485407065
F1-score:  0.8460073921182738
